Steps followed:-
1. Datest importing
2. Independent & Dependent features
3. Cleaning the data=> Stemming, StopWords
4. Fixing length of input
5. embedding layer
6. One hot encoding representation
7. constructing LSTM neural network

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('train.csv')

In [7]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [8]:
# If label is 1 -> fake news & 0 -> Not Fake news

In [9]:
df.shape

(20800, 5)

In [10]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


Since, this is a tex data. Therefore, we can not replace it with anything. Therefore dropping null rows

In [11]:
##Drop Nan Values
df.dropna(inplace=True)


In [12]:
df.shape

(18285, 5)

In [13]:
# Independent column
y = df['label']
# Dependent columns
X = df.drop(columns=['label'],axis=1)

In [14]:
X.shape

(18285, 4)

In [15]:
y.shape

(18285,)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding # help us to implement embedding, word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
tf.__version__

'2.17.1'

In [18]:
### Vocabulary size
voc_size= 5000 # -> No. of unique words we can consider.

## Onehot Representation

In [24]:
messages= X.copy()

We'll be considerig 'title' only

In [25]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [26]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [27]:
# Replacing index
messages.reset_index(inplace=True)
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


## Cleaning
### Before applying One hot encoding

In [28]:
import nltk
import re # re means regular expression. E.g we'll besing for alphabets only a-z & A-Z
from nltk.corpus import stopwords

In [29]:
# 1st we are applying StopWords
# 2nd we'll be doing stemming

# Downloading StopWords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer # Used for stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # Apart from atoz & A to Z we are replacing it by space, ' '
    review = review.lower() # Lowercasing all
    review = review.split() # Spliting word by word

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] # If word not present in 'words.('english'), Then Stem it.
    review = ' '.join(review)
    corpus.append(review)

In [31]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [32]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [36]:
# Applying One Hot Encoding
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2206, 2688, 4695, 224, 4567, 1698, 3892, 3577, 2400, 2190],
 [2694, 532, 2529, 2743, 4574, 4977, 1467],
 [280, 46, 4115, 4834],
 [2460, 2277, 3322, 1820, 3786, 1541],
 [3659, 4574, 2952, 259, 2044, 1581, 4574, 2646, 3179, 2059],
 [3678,
  4605,
  2394,
  3762,
  2449,
  2349,
  4178,
  4005,
  4542,
  3430,
  4674,
  2309,
  4448,
  3434,
  1467],
 [1511, 1370, 4162, 1749, 1639, 3855, 3245, 1268, 3956, 2300, 2317],
 [1235, 3976, 3698, 245, 3057, 2221, 2349, 223, 3956, 2300, 2317],
 [1145, 680, 2365, 1591, 3434, 145, 4416, 3673, 2349, 1935],
 [2903, 4619, 1556, 2144, 43, 3031, 2506, 2101],
 [4812, 157, 4482, 1203, 207, 117, 2921, 2381, 128, 2834, 4987],
 [1820, 3138, 4567, 145, 2349, 3057],
 [487, 193, 3959, 2391, 4823, 259, 1885, 1845, 4797],
 [4989, 1548, 4855, 1373, 4394, 432, 1363, 3956, 2300, 2317],
 [2300, 4530, 4465, 3388, 242, 3956, 2300, 2317],
 [4513, 1166, 2829, 1557, 1384, 812, 4928, 629, 4158, 3064],
 [1085, 2077, 532],
 [914, 3858, 4823, 1473, 2349, 1803, 967, 1467],
 [1

In [34]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [35]:
onehot_repr[1]

[2694, 532, 2529, 2743, 4574, 4977, 1467]

## Embedding Representation

In [41]:
# Making length of all the one hot encoding as same

In [46]:
# Selecting highest length in the text
lgt = max([len(i) for i in onehot_repr])
lgt

47

In [50]:
#Pre padding
# Making length of all the one hot encoding as same
sent_length= lgt
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2206 2688 4695 ...    0    0    0]
 [2694  532 2529 ...    0    0    0]
 [ 280   46 4115 ...    0    0    0]
 ...
 [4419 1723  872 ...    0    0    0]
 [1146 3057 4427 ...    0    0    0]
 [2551 3521 4592 ...    0    0    0]]


In [51]:
#Post padding
# Making length of all the one hot encoding as same
sent_length= lgt
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2206 2688 4695 ...    0    0    0]
 [2694  532 2529 ...    0    0    0]
 [ 280   46 4115 ...    0    0    0]
 ...
 [4419 1723  872 ...    0    0    0]
 [1146 3057 4427 ...    0    0    0]
 [2551 3521 4592 ...    0    0    0]]


Pre-padding is generally considered better than post-padding for most applications, particularly in machine learning and deep learning, for these reasons:<br>
1. Temporal Alignment: Pre-padding preserves the alignment of the original sequence by placing padding at the start. This ensures that the sequence's meaningful elements align consistently when processing sequences of different lengths.

2. Compatibility with Models: Recurrent Neural Networks (RNNs), such as LSTMs or GRUs, process sequences from the start to the end. Pre-padding keeps the later parts of the sequence (which may contain more recent or relevant information) accessible to the model without interference.

3. Ease of Truncation: When using fixed-length inputs, pre-padding allows truncation(process of shortening a sequence) from the end of the sequence, reducing the risk of discarding meaningful data at the beginning.

In [52]:
embedded_docs[0]

array([2206, 2688, 4695,  224, 4567, 1698, 3892, 3577, 2400, 2190,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

In [53]:
embedded_docs[1]

array([2694,  532, 2529, 2743, 4574, 4977, 1467,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int32)

## Creating model

In [54]:
embedding_vector_features=40 ##features representation, happens using CBOW, SkipGram
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid')) # SInce we have binaary prediction (0/1), Therefore, sigmoid.
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [55]:
len(embedded_docs),y.shape

(18285, (18285,))

In [57]:
# Converting in array
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [58]:
X_final.shape,y_final.shape

((18285, 47), (18285,))

In [59]:
# train and split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [60]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6100 - loss: 0.6286 - val_accuracy: 0.7740 - val_loss: 0.4561
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7701 - loss: 0.4660 - val_accuracy: 0.8467 - val_loss: 0.4309
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7977 - loss: 0.4840 - val_accuracy: 0.5682 - val_loss: 0.6835
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5667 - loss: 0.6846 - val_accuracy: 0.5685 - val_loss: 0.6827
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5662 - loss: 0.6841 - val_accuracy: 0.5680 - val_loss: 0.6834
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5708 - loss: 0.6832 - val_accuracy: 0.5738 - val_loss: 0.6791
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5742 - loss: 0.6810 - val_accuracy: 0.5746 - val_loss: 0.6793
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5778 - loss: 0.6794 - val_accuracy: 0

Above, we are lokkking at accuracy

## Performance Metrics And Accuracy

In [61]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [62]:
# We have kept threshold of 0.5, or else we can use :- AUC ROC Curve
y_pred=np.where(y_pred > 0.5, 1,0)


In [63]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3385,   34],
       [2489,  127]])

In [64]:
# Printing accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5819386909693455

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.99      0.73      3419
           1       0.79      0.05      0.09      2616

    accuracy                           0.58      6035
   macro avg       0.68      0.52      0.41      6035
weighted avg       0.67      0.58      0.45      6035



## Adding Dropout
* To improve the performance of the model
* Dropout is a regularization technique used to prevent overfitting in neural networks.

In [66]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3)) #In this case, 30% of the neurons in the layer will be randomly deactivated during each training step.
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [67]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.6376 - loss: 0.6084 - val_accuracy: 0.7793 - val_loss: 0.4344
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8158 - loss: 0.3888 - val_accuracy: 0.9037 - val_loss: 0.2505
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9305 - loss: 0.1925 - val_accuracy: 0.9180 - val_loss: 0.1993
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9563 - loss: 0.1267 - val_accuracy: 0.9190 - val_loss: 0.2089
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9692 - loss: 0.0982 - val_accuracy: 0.9203 - val_loss: 0.2395
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9781 - loss: 0.0768 - val_accuracy: 0.9117 - val_loss: 0.2522
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9800 - loss: 0.0707 - val_accuracy: 0.9138 - val_loss: 0.2556
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9818 - loss: 0.0673 - val_accuracy: 

In [68]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [76]:
# We have kept threshold of 0.5, or else we can use :- AUC ROC Curve
y_pred=np.where(y_pred > 0.5, 1,0)


In [77]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3159,  260],
       [ 302, 2314]])

In [78]:
# Printing accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9068765534382767

In [79]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.90      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [ ]:
# UCI repository dataset:- Lots of problem statements, try solving any text classification